In [10]:
"""
Provide features used in speaker separation. 
Provides: MFCC

Author: Ren Yuan (Peter) Xue
"""

import numpy as np
import functions as func
from scipy.fftpack import dct

def mfcc(signal, sample_rate=None, pre_emph=0.97, frame_size=0.025, frame_step=0.01,
         num_fft=None, freq_min=0, freq_max=None, num_mel_filter=26, norm=False):
    """
    Function mfcc calculates the MFCC feature from an audio signal.
    
    @param signal: Audio signal.
    @param sample_rate: The sample rate of audio signal we are working with.
    @param pre_emph: Coefficient used in pre-empahsis filter. Default is 0.97.
    @param frame_size: Time interval we are taking within frames. Default is 0.025.
    @param frame_step: Time step we are taking between frames. Default is 0.01.
    @param num_fft: Number of FFT points. Default is calculated using get_num_fft.
    @param freq_min: Lowest frequency band edge of Mel filters in Hz. Default is 0.
    @param freq_max: Highest frequency band edge of Mel filters in Hz. Default is sample rate / 2.
    @param num_mel_filter: Number of filter points in filter banks on Mel scale.
    @param norm: Whether or not perform mean normalization. Default is False.
    @returns: The MFCC feature from audio signal.
    """
    # Handle exceptions.
    if not sample_rate: # Check samplerate input validness.
        return('Invalid input for sample_rate')
    if freq_max  and freq_max > sample_rate / 2: # Check maximum frequency input validness.
        return('Invalid input for freq_max')
    filter_banks = log_filter_bank(signal, sample_rate, pre_emph, frame_size, frame_step,
                              num_fft, freq_min, freq_max, num_mel_filter, False)
    mfcc = dct(filter_banks, type = 2, axis = 1, norm = 'ortho')
    return mfcc if not norm else mfcc - np.mean(mfcc, axis = 0) + 1e-8
    
    
def filter_bank(signal, sample_rate=None, pre_emph=0.97, frame_size=0.025, frame_step=0.01,
         num_fft=None, freq_min=0, freq_max=None, num_mel_filter=26, norm=False):
    """
    Function filter_bank calculates the filter bank feature from an audio signal.
    
    @param signal: Audio signal.
    @param sample_rate: The sample rate of audio signal we are working with.
    @param pre_emph: Coefficient used in pre-empahsis filter. Default is 0.97.
    @param frame_size: Time interval we are taking within frames. Default is 0.025.
    @param frame_step: Time step we are taking between frames. Default is 0.01.
    @param num_fft: Number of FFT points. Default is calculated using get_num_fft.
    @param freq_min: Lowest frequency band edge of Mel filters in Hz. Default is 0.
    @param freq_max: Highest frequency band edge of Mel filters in Hz. Default is sample rate / 2.
    @param num_mel_filter: Number of filter points in filter banks on Mel scale.
    @param norm: Whether or not perform mean normalization. Default is False.
    @returns: The filter bank feature from audio signal.
    """
    # Handle exceptions.
    if not sample_rate: # Check samplerate input validness.
        return('Invalid input for sample_rate')
    if freq_max  and freq_max > sample_rate / 2: # Check maximum frequency input validness.
        return('Invalid input for freq_max')
    
    # Initialze variables.
    num_fft = num_fft or func.get_num_fft(sample_rate, frame_size)
    freq_max = freq_max or int(np.floor(sample_rate / 2))
    # Apply pre-emphasize filter to audio.
    emphasized_signal = func.pre_emphasis(signal, pre_emph)
    # Frame audio and apply Hamming window.
    frames = func.frame_audio(emphasized_signal, sample_rate, frame_size, frame_step, num_fft) 
    frames *= np.hamming(int(round(sample_rate * frame_size)))
    # Calculate the Power Spectrum of the frames.
    magnitude_frames = np.absolute(np.fft.rfft(frames, num_fft))
    power_frames = ((1.0 / num_fft) * (magnitude_frames) ** 2)
    # Construct filter_banks.
    print([freq_min,freq_max,num_mel_filter,num_fft,sample_rate])
    filters = func.get_filter(freq_min,freq_max,num_mel_filter,num_fft,sample_rate) # Construct filters.
    print(filters)
    filter_banks = np.dot(filters, power_frames.T)
    filter_banks = np.where(filter_banks == 0, np.finfo(float).eps, filter_banks) # Numerical stability
    # Return the filter bank based on mean normalization = True or False.
    return filter_banks if not norm else filter_banks - np.mean(filter_banks, axis = 0) + 1e-8 


def log_filter_bank(signal, sample_rate=None, pre_emph=0.97, frame_size=0.025, frame_step=0.01,
         num_fft=None, freq_min=0, freq_max=None, num_mel_filter=26, norm=False):
    """
    Function log_filter_bank calculates the filter bank feature from an audio signal.
    
    @param signal: Audio signal.
    @param sample_rate: The sample rate of audio signal we are working with.
    @param pre_emph: Coefficient used in pre-empahsis filter. Default is 0.97.
    @param frame_size: Time interval we are taking within frames. Default is 0.025.
    @param frame_step: Time step we are taking between frames. Default is 0.01.
    @param num_fft: Number of FFT points. Default is calculated using get_num_fft.
    @param freq_min: Lowest frequency band edge of Mel filters in Hz. Default is 0.
    @param freq_max: Highest frequency band edge of Mel filters in Hz. Default is sample rate / 2.
    @param num_mel_filter: Number of filter points in filter banks on Mel scale.
    @param norm: Whether or not perform mean normalization. Default is False.
    @returns: The log filter bank feature from audio signal.
    """
    # Handle exceptions.
    if not sample_rate: # Check samplerate input validness.
        return('Invalid input for sample_rate')
    if freq_max  and freq_max > sample_rate / 2: # Check maximum frequency input validness.
        return('Invalid input for freq_max')
    
    # Initialze variables.
    num_fft = num_fft or func.get_num_fft(sample_rate, frame_size)
    freq_max = freq_max or int(np.floor(sample_rate / 2))
    # Compute filter bank from function filter_bank.
    filter_banks = filter_bank(signal, sample_rate, pre_emph, frame_size, frame_step,
                              num_fft, freq_min, freq_max, num_mel_filter, False)
    filter_banks = np.log(filter_banks) # Take log.
    return filter_banks if not norm else filter_banks - np.mean(filter_banks, axis = 0) + 1e-8 

In [12]:
import os
import sys
import numpy as np
import scipy
from scipy.io import wavfile
import scipy.fftpack as fft
from scipy.signal import get_window
import IPython.display as ipd
import matplotlib.pyplot as plt

%matplotlib inline

path = '/Users/Vicissitudes/Documents/MFCC/'
sample_rate, audio = wavfile.read(path + "test.wav")
#filt = filter_bank(signal = audio[:,0], sample_rate = sample_rate)
#filt.shape
filt_2 = mfcc(signal = audio[:,0], sample_rate = sample_rate)
filt_2.shape

[0, 4000, 26, 256, 8000]
[[0.         1.         0.         ... 0.         0.         0.        ]
 [0.         0.         1.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.11111111 0.         0.        ]]


(26, 3351)

In [ ]:
def main():
    print('hello world')
if __name__ == '__main__':
    main()